In [2]:
import pandas as pd
import plotly.express as px


In [3]:
df= pd.read_csv('data/datos-2026-02-03.csv')
df.head()

,fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica
0,2017-05-06,Robo a carga pesada,NaN,NaN,NO DISPONIBLE,TONALA,101.0,17:45,El patrimonio,AMG
1,2017-05-30,Robo a vehiculos particulares,NaN,NaN,NO DISPONIBLE,ENCARNACION DE DIAZ,35.0,5:30,El patrimonio,Interior
2,2017-08-08,Homicidio doloso,NaN,NaN,NO DISPONIBLE,TLAJOMULCO DE ZUÑIGA,97.0,17:45,La vida y la integridad corporal,AMG
3,2017-08-09,Robo casa habitacion,NaN,NaN,NO DISPONIBLE,LAGOS DE MORENO,53.0,17:45,El patrimonio,Interior
4,2017-12-28,Robo de motocicleta,NaN,NaN,NO DISPONIBLE,LA BARCA,18.0,5:30,El patrimonio,Interior


In [4]:
df["fecha"] = pd.to_datetime(df["fecha"])
df = df.sort_values("fecha")


In [5]:
df["año"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month

In [6]:
df.head()

,fecha,delito,x,y,colonia,municipio,clave_mun,hora,bien_afectado,zona_geografica,año,mes
16883,2017-01-01,Lesiones dolosas,-103.284071,20.599576,SAN MARTIN DE LAS FLORES DE ABAJO,SAN PEDRO TLAQUEPAQUE,98.0,17:45,La vida y la integridad corporal,AMG,2017,1
370,2017-01-01,Robo a negocio,NaN,NaN,NO DISPONIBLE,ZAPOPAN,120.0,5:30,El patrimonio,AMG,2017,1
18734,2017-01-01,Robo a negocio,-103.370416,20.668334,OBRERA,GUADALAJARA,39.0,5:30,El patrimonio,AMG,2017,1
16285,2017-01-01,Robo de motocicleta,-103.274508,20.656566,TETLAN II,GUADALAJARA,39.0,5:30,El patrimonio,AMG,2017,1
15474,2017-01-01,Robo casa habitacion,-103.399580,20.637874,LOMA BONITA,ZAPOPAN,120.0,5:30,El patrimonio,AMG,2017,1


In [20]:
delitos_interes = [
    "Robo a negocio",
    "Robo a persona",
    "Robo a vehiculos particulares",
    "Violencia familiar",
]
df_filtrado = df[df["delito"].isin(delitos_interes)]

conteo_anual = (
    df_filtrado
    .groupby(["año", "delito"])
    .size()
    .reset_index(name="casos")
)


fig = px.bar(
    conteo_anual,
    x="año",
    y="casos",
    color="delito",
    barmode="group",
    title="Casos por tipo de delito y año en Jalisco"
)

fig.show()

In [16]:
year= [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024,2025]

for y in year:
    df_year = (
        df[df["año"] == y]
        .groupby("delito")
        .size()
        .reset_index(name="casos")
    )
    fig = px.bar(
        df_year,
        x="delito",
        labels={"delito": "Tipo de delito" , "casos": "Número de casos"},
        y="casos",
        title=f"Casos por tipo de delito en {y} en Jalisco"
    )

    fig.show()


In [9]:
fig = px.line(
    conteo_anual,
    x="año",
    y="casos",
    color="delito",
    markers=True,
    title="Evolución de delitos en Jalisco (2017–2025)"
)

fig.show()

In [10]:
pivot = conteo_anual.pivot(
    index="año",
    columns="delito",
    values="casos"
)

crecimiento = pivot.pct_change() * 100
crecimiento.reset_index(inplace=True)

In [11]:
fig = px.line(
    crecimiento,
    x="año",
    y=crecimiento.columns[1:],  # delitos
    title="Variación porcentual anual de delitos"
)

fig.show()

In [12]:
top_colonias = (
    df[df["colonia"] != "NO DISPONIBLE"]
    .groupby("colonia")
    .size()
    .sort_values(ascending=False)
    .head(10)
    .reset_index(name="casos")
)

fig = px.bar(
    top_colonias,
    x="casos",
    y="colonia",
    orientation="h",
    title="Colonias con mayor número de delitos"
)

fig.show()

In [13]:
df_2024 = df[df["año"] == 2024]

fig = px.pie(
    df_2024,
    names="delito",
    title="Distribución de delitos en 2024"
)

fig.show()

In [14]:
por_municipio = (
    df
    .groupby("municipio")
    .size()
    .reset_index(name="casos")
)